In [ ]:
from rag_tests.qdrant_db import QdrantDatabaseConnector
from rag_tests.config import settings
from dotenv import load_dotenv
from rag_tests.llm.chain import GeneralQueryChain
from rag_tests.rag.retreiver import VectorRetriever
from langchain_openai import ChatOpenAI
from rag_tests.config import settings
from rag_tests.llm.prompt_templates import GetToneUnderstandingPromptTemplate, ExtractToneFromHitsPromptTemplate

## PROMPTS

## GENERATION FLOW

In [ ]:
TOPIC = """what are common mistakes that entrepreneurs make??."""
AUTHOR_ID = "https://www.linkedin.com/in/varunaggarwal2"

In [ ]:
collection_name = f"TEST_VARUN_{settings.EMBEDDING_MODEL_ID.replace('/','_')}"

In [ ]:
load_dotenv()
print("Starting the process")

author_id = "https://www.linkedin.com/in/varunaggarwal2"
query = f"""
    My author_id is https://www.linkedin.com/in/varunaggarwal2
    i want to draft a LinkedIn post discussing {TOPIC}.
    """
retriever = VectorRetriever(query=query)
hits = retriever.retrieve_top_k(k=6, to_expand_to_n_queries=5, collection_name=collection_name)
print(f"Retrieved hits: {hits}")



In [ ]:
[{"Post " + str(i): hit.payload['content']} for i, hit in enumerate(hits)]

In [ ]:
# Extracting tone from all hits
model = ChatOpenAI(model=settings.OPENAI_MODEL_ID, temperature=0)   
get_tone_understanding_prompt_template = ExtractToneFromHitsPromptTemplate()
prompt_template = get_tone_understanding_prompt_template.create_template()
chain = GeneralQueryChain().get_chain(
    llm=model, output_key="tone_understanding", template=prompt_template
)
response = chain.invoke({"author_id":AUTHOR_ID, "posts":[{"Post " + str(i): hit.payload['content']} for i, hit in enumerate(hits)]})
print(f"Get Tone Understanding.generate_response() result: \033[93m{response['tone_understanding']}\033[0m")

In [ ]:
print(response['tone_understanding'])

In [ ]:
reranked_hits = retriever.rerank(hits=hits, keep_top_k=5)
for rank, hit in enumerate(reranked_hits):
    print(f"{rank}: {hit}")

In [ ]:

prompt = f"""
You are a helpful assistant that can draft a LinkedIn post.
USING ONLY THE FOLLOWING tone, patterns and voice of the author {AUTHOR_ID} extracted from similar posts to the current topic, draft me a LinkedIn post discussing {TOPIC}.

The tone, patterns and voice of the author is: {response['tone_understanding']}
"""
response = model.invoke(prompt)
print(f"Response: \033[93m{response.content}\033[0m")

In [ ]:
print(response.content)